In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install statsforecast


In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import MSTL, AutoARIMA

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# read the data
train = pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv')
test = pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv")
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e19/sample_submission.csv')

In [ ]:
#create a test dataframe copy
test_copy = test

In [ ]:
train.head()

In [ ]:
plt.rcParams["figure.figsize"] = (9,6)

In [ ]:
#shape of train and test.
train.shape, test.shape

In [ ]:
train['date'].describe()

In [ ]:
train['date'] = pd.to_datetime(train['date'])

In [ ]:
train.info()

In [ ]:
train['unique_id'] = train.groupby(['country', 'store', 'product'], sort = False).ngroup()+1
train.unique_id.unique()

In [ ]:
# set the date to be the index but we can see it is repeated 
train.set_index('date')

In [ ]:
# remove column id from the dataset
train = train.loc[:,train.columns !='id']

In [ ]:
train.rename(columns= {'date': 'ds', 'num_sold':'y'}, inplace=True)
train.head()

In [ ]:
train = train.drop(['country', 'store', 'product'], axis= 1)
train.head()

In [ ]:
StatsForecast.plot(train)

In [ ]:
# adding exogenous regresor COVID19 in the durationa from Februery 2020 to July 2020
train['COVID19'] = np.where((pd.to_datetime(train['ds']) > '2022-02-01')& (pd.to_datetime(train['ds']) < '2022-08-01') , 1, 0)

In [ ]:
train.head()

Use MSTL model for the multiple seasonality

In [ ]:
# Create an MSTL model  and instantiation parameters

models = [MSTL(
    season_length=[24, 24 * 7], # seasonalities of the time series weekly and annual
    trend_forecaster=AutoARIMA() # model used to forecast trend
)]


In [ ]:
# StatsForecast MSTL model with weekly and annual seasonality
sf_mstl = StatsForecast(
    models = models,
    freq = 'D', n_jobs=-1
)

In [ ]:
sf_mstl.fit(train)

In [ ]:
# decompose the model 

sf_mstl.fitted_[0, 0].model_

# Test dataframe

In [ ]:
test['unique_id'] = test.groupby(['country', 'store', 'product'], sort = False).ngroup()+1

In [ ]:
# set the date to be the index but we can see it is repeated 
test.set_index('date')

In [ ]:
# remove column id from the dataset
test = test.loc[:,test.columns !='id']

In [ ]:
test.rename(columns= {'date': 'ds', 'num_sold':'y'}, inplace=True)
test.head()

In [ ]:
test = test.drop(['country', 'store', 'product'], axis= 1)
test.head()

In [ ]:
# Adding COVID 19 for the test dataframe to allow the forecasting function to use it 
test['COVID19'] = np.where((pd.to_datetime(test['ds']) > '2022-02-01')& (pd.to_datetime(test['ds']) < '2022-08-01') , 1, 0)
test.head()

In [ ]:
forecasts = sf_mstl.forecast(df = train, h=365, X_df = test)
forecasts.head()

In [ ]:
sf_mstl.plot(train, forecasts, level = [90])

In [ ]:
forecasts = forecasts.reset_index()
forecasts.head()

In [ ]:
forecasts.info()

In [ ]:
test.info()

In [ ]:
test_copy['date'] = pd.to_datetime(test_copy['date'])

In [ ]:
final = forecasts.merge(test_copy, how = 'left', left_on = ['unique_id', 'ds'], right_on = ['unique_id','date'])
final.head()

In [ ]:
submission = final[['id', 'MSTL']].sort_values('id').reset_index(drop = True)
submission.rename(columns={"MSTL": "num_sold"}, inplace = True)
submission['num_sold'] = submission['num_sold'].astype(int) 
submission.head()

In [ ]:
submission.info()

In [ ]:
submission.to_csv('submission.csv', index = False)